In [ ]:
import torch
import torch.optim as optim
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


# Create a dataset

In [ ]:
df = pd.DataFrame({'Lecture Hours' : [1, 2, 3, 4, 5, 6],
                   'Lab Hours' : [2, 3, 1, 3, 3, 2],
                   'Label' : ['Fail', 'Fail', 'Fail', 'Pass', 'Pass', 'Pass']})
df

,Lecture Hours,Lab Hours,Label
0,1,2,Fail
1,2,3,Fail
2,3,1,Fail
3,4,3,Pass
4,5,3,Pass
5,6,2,Pass


In [ ]:
df.dtypes

Lecture Hours     int64
Lab Hours         int64
Label            object
dtype: object

In [ ]:
df['Label'].replace({'Fail': 0, 'Pass': 1}, inplace=True)
df

,Lecture Hours,Lab Hours,Label
0,1,2,0
1,2,3,0
2,3,1,0
3,4,3,1
4,5,3,1
5,6,2,1


In [ ]:
x_data = df.iloc[:, :-1].to_numpy() #independent variables
y_data = df.iloc[:, [-1]].to_numpy() #dependent variable
x_data

array([[1, 2],
       [2, 3],
       [3, 1],
       [4, 3],
       [5, 3],
       [6, 2]])

In [ ]:
y_data

array([[0],
       [0],
       [0],
       [1],
       [1],
       [1]])

In [ ]:
x = torch.FloatTensor(x_data) #Tensor for independent variables
y = torch.FloatTensor(y_data) #Tensor for dependent variable

w = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# hypothesis = 1 / (1 + torch.exp(-(x.matmul(w) + b)))
hypothesis = torch.sigmoid(x.matmul(w) + b)
print(hypothesis)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)


In [ ]:
cost = -(y*torch.log(hypothesis) + (1-y)*torch.log(1-hypothesis)).mean()
print(cost)

tensor(0.6931, grad_fn=<NegBackward0>)


In [ ]:
import torch.nn.functional as F

optimizer = optim.SGD([w, b], lr = 1.0) #1e-1 = 0.1

for epoch in range(1001):
  hypothesis = torch.sigmoid(x.matmul(w) + b)
  # cost = -(y*torch.log(hypothesis) + (1-y)*torch.log(1-hypothesis)).mean()
  cost = F.binary_cross_entropy(hypothesis, y)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print("Epoch: {:4d} / {} Cost: {:.4f}".format(epoch, 1000, cost.item()))

Epoch:    0 / 1000 Cost: 0.0183
Epoch:  100 / 1000 Cost: 0.0169
Epoch:  200 / 1000 Cost: 0.0156
Epoch:  300 / 1000 Cost: 0.0146
Epoch:  400 / 1000 Cost: 0.0137
Epoch:  500 / 1000 Cost: 0.0128
Epoch:  600 / 1000 Cost: 0.0121
Epoch:  700 / 1000 Cost: 0.0115
Epoch:  800 / 1000 Cost: 0.0109
Epoch:  900 / 1000 Cost: 0.0104
Epoch: 1000 / 1000 Cost: 0.0099


In [ ]:
print(hypothesis[:6])

tensor([[4.9076e-05],
        [1.6176e-02],
        [1.9336e-02],
        [9.7737e-01],
        [9.9955e-01],
        [9.9994e-01]], grad_fn=<SliceBackward0>)


In [ ]:
print(format(4.9076e-05, 'f'))

0.000049


In [ ]:
print(format(9.7737e-01, 'f'))

0.977370


In [ ]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:6])
print(y[:6])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [ ]:
correct_prediction = prediction.float() == y
print(correct_prediction)

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True]])


In [ ]:
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print("The model has an accuracy of {:.2f} with the given dataset".format(accuracy*100))

The model has an accuracy of 100.00 with the given dataset


# TensorFlow

In [ ]:
x = tf.placeholder(tf.float32, shape=[6, 2])
y = tf.placeholder(tf.float32, shape=[6, 1])

w = tf.Variable(tf.random_normal([2, 1], name="weight"))
b = tf.Variable(tf.random_normal([1], name="bias"))

hypothesis = tf.sigmoid(tf.matmul(x, w) + b)
cost = -tf.reduce_mean(y * tf.log(hypothesis) + (1-y) * tf.log(1-hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate = 1.0).minimize(cost)

prediction = tf.cast(hypothesis >= 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y), dtype=tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(1001):
  _, cost_val = sess.run([train, cost], feed_dict={x: x_data, y: y_data})
  if epoch % 100 == 0:
    print(epoch, cost_val)

  h, c, a = sess.run([hypothesis, prediction, accuracy], feed_dict={x: x_data, y: y_data})
  print("\nHypothesis: ", h, "\nPrediction (Y): ", c, "\nAccuracy: ", a)

Streaming output truncated to the last 5000 lines.
 [4.5141440e-02]
 [5.7458613e-02]
 [9.3795824e-01]
 [9.9631476e-01]
 [9.9923033e-01]] 
Prediction (Y):  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  1.0

Hypothesis:  [[7.0702418e-04]
 [4.5084991e-02]
 [5.7378665e-02]
 [9.3803406e-01]
 [9.9632436e-01]
 [9.9923277e-01]] 
Prediction (Y):  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  1.0

Hypothesis:  [[7.0466049e-04]
 [4.5028612e-02]
 [5.7298921e-02]
 [9.3810958e-01]
 [9.9633384e-01]
 [9.9923521e-01]] 
Prediction (Y):  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  1.0

Hypothesis:  [[7.0230756e-04]
 [4.4972342e-02]
 [5.7219330e-02]
 [9.3818492e-01]
 [9.9634337e-01]
 [9.9923760e-01]] 
Prediction (Y):  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  1.0

Hypothesis:  [[6.9996627e-04]
 [4.4916257e-02]
 [5.7140108e-02]
 [9.3826014e-01]
 [9.9635279e-01]
 [9.9924004e-01]] 
Prediction (Y):  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  1.0

Hypothesis:  [[6.9763575e-04]
 [4.48603